## Competición del titanic-kaggle

En líneas generales, la idea con la competición es construir un modelo básico de Machine Learnign capaz de predecir la propabilidad, en términos de unos y ceros, que tienen los distintos pasajeros del titanic de salvarse (cada pasajero está identificado de manera única).

## Se cuentan con dos archivos iniciales:
* `test.csv`
* `train.csv`

A partir de estos archivos, debemos generar un CSV que sea capaz responda la pregunta de la competición, utilizando la estructura antes dicha, que se resume en **`0` para los que no se salvarán y `1` para los que sí lo harán**.

# Información del grupo:
* Deivy Peña - 1099429.
* Juan Avila - 1100378.
* Vladimir González - 1098139.
* Leonel Rodríguez - 1098665.


# 1. Importando librerías

In [9]:
import pandas as pd

# Preprocesamiento, refrecion, split para separar las variables a utilizar y, por último, el accuracy_score para determinar las respuestas en el formato dado.
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 2. Cargando y visualizando la data de manera inizial

In [16]:
print("Data inicial:")
data = pd.read_csv("train.csv")
data.head()

Data inicial:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
print("Data de prueba inicial:")
test = pd.read_csv("test.csv")
test.head()

Data de prueba inicial:


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# 3. Extrayendo los ids de los pasajeros y creando la función clean

In [2]:
test_ids = test["PassengerId"]
def clean(data):
    data = data.drop(["Ticket", "PassengerId", "Name", "Cabin"], axis=1)
    
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
        
    data.Embarked.fillna("U", inplace=True)
    return data
data = clean(data)
test = clean(test)

In [3]:
data.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S


In [5]:
le = preprocessing.LabelEncoder()
columns = ["Sex", "Embarked"]

for col in columns:
    data[col] = le.fit_transform(data[col])
    test[col] = le.transform(test[col])
    print(le.classes_)
      
data.head(5)

['female' 'male']
['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [7]:
y = data["Survived"]
X = data.drop("Survived", axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)

In [10]:
predictions = clf.predict(X_val)
accuracy_score(y_val, predictions)

0.8100558659217877

In [14]:
submission_preds = clf.predict(test)

In [12]:
df = pd.DataFrame({"PassengerId": test_ids.values,
                   "Survived": submission_preds,
                  })

In [ ]:
df.to_csv("submission.csv", index=False)